# Secondary Drift Detection for Simple Configuration

## Setup
### Imports

In [7]:
# imports
import helper
from pm4py.objects.log.importer.xes import importer as xes_importer

from processdrift.framework import drift_detection
from processdrift.framework import drift_explanation
from processdrift.framework import feature_extraction
from processdrift.framework import population_comparison
from processdrift.framework import windowing
from processdrift.framework import evaluation
from processdrift.framework import change_point_extraction


import time
import os
import math

### Data settings and event log loading

In [2]:
# initial settings
input_path = r'data\synthetic\attribute_drift\sudden_3_attribute_values\10000'

results_path = r'results\sudden_3_attribute_values\10000\100_iter_results.csv'
delete_if_results_exists = True

experiment_name = 'initial'

# settings
window_generator_types = ['fixed', 'adaptive']
window_sizes= [100, 150, 200]
thresholds = [0.05]
max_distances = [300]
slide_bys = [10, 20] # , 10]
proportional_phis = [0.25, 0.5, 1]
proportional_rhos = [0.05]

# limit iterations
limit_iterations = None

In [3]:
configurations = helper.get_configurations(window_generator_types, window_sizes, thresholds, max_distances, slide_bys, proportional_phis, proportional_rhos)
print(len(configurations))
helper.perform_synthetic_experiments(experiment_name, configurations, input_path, results_path, delete_if_results_exist=delete_if_results_exists, limit_iterations=limit_iterations)

36
Event log 0


c:\Users\Lennart\Tools\Anaconda3\envs\concept_drift\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 10000/10000 [00:08<00:00, 1138.16it/s]



Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 0.25, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 0.5, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 1, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 0.25, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 0.5, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed',

parsing log, completed traces :: 100%|██████████| 10000/10000 [00:11<00:00, 892.06it/s]



Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 0.25, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 0.5, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 1, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 0.25, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 0.5, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed',

...


Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 0.25, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 0.5, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 1, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 0.25, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 0.5, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed',

parsing log, completed traces :: 100%|██████████| 10000/10000 [00:08<00:00, 1195.75it/s]



Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 0.25, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 0.5, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 1, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 0.25, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 0.5, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed',

parsing log, completed traces :: 100%|██████████| 10000/10000 [00:07<00:00, 1286.25it/s]



Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 0.25, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 0.5, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 1, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 0.25, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 0.5, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed',

In [4]:
# perform more tests for different slide by settings
input_path = r'data\synthetic\attribute_drift\sudden_3_attribute_values\10000'

results_path = r'results\sudden_3_attribute_values\10000\100_iter_results.csv'
delete_if_results_exists = False

experiment_name = 'slide_bys'

# settings
window_generator_types = ['fixed']
window_sizes= [200]
thresholds = [0.05]
max_distances = [300]
slide_bys = [1, 5, 10, 15, 20, 25, 30, 35, 40]
proportional_phis = [1]
proportional_rhos = [0.1]

# limit iterations
limit_iterations = None

In [5]:
configurations = helper.get_configurations(window_generator_types, window_sizes, thresholds, max_distances, slide_bys, proportional_phis, proportional_rhos)
print(len(configurations))
helper.perform_synthetic_experiments(experiment_name, configurations, input_path, results_path, delete_if_results_exist=delete_if_results_exists, limit_iterations=limit_iterations)

9
Event log 0


parsing log, completed traces :: 100%|██████████| 10000/10000 [00:08<00:00, 1236.99it/s]



Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 1, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 5, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 15, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 2

parsing log, completed traces :: 100%|██████████| 10000/10000 [00:07<00:00, 1298.49it/s]



Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 1, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 5, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 15, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 2

...


Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 1, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 5, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 15, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 2

parsing log, completed traces :: 100%|██████████| 10000/10000 [00:08<00:00, 1216.79it/s]



Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 1, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 5, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 15, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 2

parsing log, completed traces :: 100%|██████████| 10000/10000 [00:07<00:00, 1269.92it/s]



Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 1, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 5, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 15, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 200, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 1, 'proportional_rho': 0.1}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 2

In [8]:
# initial settings
input_path = r'data\synthetic\attribute_drift\sudden_10_attribute_values\10000'

results_path = r'results\sudden_10_attribute_values\10000\100_iter_results.csv'
delete_if_results_exists = True

experiment_name = 'initial'

# settings
window_generator_types = ['fixed', 'adaptive']
window_sizes= [100, 150, 200]
thresholds = [0.05]
max_distances = [300]
slide_bys = [10, 20] # , 10]
proportional_phis = [0.25, 0.5, 1]
proportional_rhos = [0.05]

# limit iterations
limit_iterations = None

In [9]:
configurations = helper.get_configurations(window_generator_types, window_sizes, thresholds, max_distances, slide_bys, proportional_phis, proportional_rhos)
print(len(configurations))
helper.perform_synthetic_experiments(experiment_name, configurations, input_path, results_path, delete_if_results_exist=delete_if_results_exists, limit_iterations=limit_iterations)

36
Event log 0


parsing log, completed traces :: 100%|██████████| 10000/10000 [00:14<00:00, 708.74it/s]



Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 0.25, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 0.5, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 10, 'proportional_phi': 1, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 0.25, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed', 'window_size': 100, 'threshold': 0.05, 'max_distance': 300, 'slide_by': 20, 'proportional_phi': 0.5, 'proportional_rho': 0.05}

Evaluating configuration {'window_generator_type': 'fixed',

In [ ]:
import winsound
for i in range(1, 100):
    winsound.Beep(i*100, 200)

In [ ]:
# initial settings
input_path = r'data\synthetic\attribute_drift\recurring_3_attribute_values\10000'

results_path = r'results\recurring_3_attribute_values\10000\100_iter_results.csv'
delete_if_results_exists = True

experiment_name = 'initial'

# settings
window_generator_types = ['fixed', 'adaptive']
window_sizes= [100, 150, 200]
thresholds = [0.05]
max_distances = [300]
slide_bys = [10, 20] # , 10]
proportional_phis = [0.25, 0.5, 1]
proportional_rhos = [0.05]

# limit iterations
limit_iterations = None

In [ ]:
configurations = helper.get_configurations(window_generator_types, window_sizes, thresholds, max_distances, slide_bys, proportional_phis, proportional_rhos)
print(len(configurations))
helper.perform_synthetic_experiments(experiment_name, configurations, input_path, results_path, delete_if_results_exist=delete_if_results_exists, limit_iterations=limit_iterations)

36
Event log 0


parsing log, completed traces ::  29%|██▉       | 2924/10000 [00:04<00:08, 852.21it/s]

In [ ]:
import winsound
for i in range(1, 100):
    winsound.Beep(i*100, 200)